In [1]:
import os
import json
import pandas as pd
from collections import defaultdict

In [2]:
common_col = ['ID', 'Tissue', 'Disease', 'Condition', 'Age', 'Age_unit', 'Gender', 'Race']

In [17]:
import os
import json
import pandas as pd
from collections import defaultdict

common_col = ['ID', 'Tissue', 'Disease', 'Condition', 'Age', 'Age_unit', 'Gender', 'Race']
code_mapping_dir = '../Datasets/mapped_lv3'
hand_mapping_dir = '../Datasets/meta'
version = 3

record_diff = defaultdict(dict)
table = {key: [] for key in ['dataset']+common_col}
for i, file_name in enumerate(os.listdir(hand_mapping_dir)):
    dataset = file_name.split('_')[0]
    hand_mapping_file_path = os.path.join(hand_mapping_dir, file_name)
    code_mapping_file_path = os.path.join(code_mapping_dir, dataset+'_pheno.csv')
    try:
        code_mapping_data = pd.read_csv(code_mapping_file_path)
    except:
        continue
    if file_name.split('.')[-1] != 'csv': 
        hand_mapping_data = pd.read_excel(hand_mapping_file_path)
    else:
        hand_mapping_data = pd.read_csv(hand_mapping_file_path)
    table['dataset'].append(dataset)
    for col in common_col:
        code_made = set([str(item) for item in code_mapping_data[col] if str(item) != 'nan'])
        hand_made = set([str(item) for item in hand_mapping_data[col] if str(item) != 'nan'])
        hand_diff = hand_made.difference(code_made)
        code_diff = code_made.difference(hand_made)
        if not hand_diff and not code_diff: 
            table[col].append('*')
            continue
        table[col].append(' ')
        record_diff[dataset][col] = {'hand_made': sorted(list(hand_diff)), 
                                    'code_made': sorted(list(code_diff))
                                    }
df = pd.DataFrame(table)
df.to_csv('complement_table_v{}.csv'.format(version))

with open('Compare_TwoVesion_HeadingFile_v{}.json'.format(version), 'w') as jf:
    json.dump(record_diff, jf, indent=4, ensure_ascii=False)